# Multilevel and marginal modeling, a case study with the NHANES data

DRAFT WORK IN PROGRESS
======================

This notebook is a counterpart to our introductory regression modeling
case study for independent data using the NHANES data.  Here we will
build on the basic linear and logistic regression approaches discussed
previously in the setting of independent data.  We will be exploring
the use of some more advanced regression approaches that can be used
for data that are statistically dependent.

We begin by importing the libraries that we will be using.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import statsmodels.api as sm
import numpy as np

Data can be *dependent*, or have a *mutilevel structure* for many
reasons.  In practice, most datasets exhibit some form of dependence,
and it is arguably independent data, not dependent data, that should
be treated as the exceptional case. Here we will reconsider the NHANES
data from the perspective of dependence, focusing in particular on
dependence in the data that arises due to *clustering* which we will
define below.

First, we read in the data, as we have done before.  For simplicity,
we remove all rows of data with missing values in the variables of
interest (note that there are more sophisticated approaches for
handling missing data that generally will give better results, but for
simplicity we do not use them here).

Note that we retain two variables here
[SDMVSTRA](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#SDMVSTRA)
and
[SDMVPSU](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#SDMVPSU)
that will be used below to define the clustering structure in these
data.

In [2]:
url = "https://raw.githubusercontent.com/kshedden/statswpy/master/NHANES/merged/nhanes_2015_2016.csv"
da = pd.read_csv(url)

# Drop unused columns, drop rows with any missing values.
vars = ["BPXSY1", "RIDAGEYR", "RIAGENDR", "RIDRETH1", "DMDEDUC2", "BMXBMI",
        "SMQ020", "SDMVSTRA", "SDMVPSU"]
da = da[vars].dropna()

## Introduction to clustered data

One common reason that data are dependent is that the data values were
collected in clusters.  This essentially means that the population was
partitioned into groups, a limited number of these groups were somehow
selected, and then a limited number of individuals were selected from
each of the selected groups.  In a proper survey, there is a
well-planned design, in which both the groups, and the individuals
within groups, are selected randomly.  The goal in doing this is to
maximize the chances that the sample is representative of the
population of interest in all relevant ways.  But many other data sets
exhibit clustering structure, even when the data collection was not so
carefully planned.

Regardless of how the clustering in the sample arose, it is likely to
be the case that observations within a cluster are more similar to
observations in different clusters.  To make this concrete, note that
clustering is often geographic.  Data may be collected by visiting
several locations, then recruiting participants within each location.
People within a location may share similarities, for example in
demography and lifestyle, or they may share environmental
circumstances such as climate.  When we have clustered data, it is
usually advisable to account for this in the analysis.

## Clustering structure in NHANES

The detailed process of collecting data for a study like NHANES is
very complex, so we will simplify things substantially here (more
details can be found
[here](https://wwwn.cdc.gov/nchs/nhanes/analyticguidelines.aspx) but
are not needed for this course). Roughly speaking, in NHANES the data
are collected by selecting a limited number of counties in the US,
then selecting subregions of these counties, then selecting people
within these subregions.  Since counties are geographically
constrained, it is expected that people within a county are more
similar to each other than they are to people in other counties.

If we could obtain the county id where each NHANES participant
resides, we could directly study this clustering structure.  However
for privacy reasons this information is not released with the data.
Instead, we have access to "masked variance units" (MVUs), which are
formed by combining subregions of different counties into artificial
groups that are not geographically contiguous.  While the MVUs are not
the actual clusters of the survey, and are not truly contiguous
geographic regions, they are deliberately selected to mimic these
things, while minimizing the risk that a subject can be "unmasked" in
the data.  For the remainder of this notebook, we will treat the MVUs
as clusters, and explore the extent to which they induce correlations
in some of the NHANES variables that we have been studying.

The MVU identifiers can be obtained by combining the `SDMVSTRA` and
`SDMVPSU` identifiers, which we do next:

In [3]:
da["group"] = 10*da.SDMVSTRA + da.SDMVPSU

## Intraclass correlation

Similarity among observations within a cluster can be measured using a
statistic called the *intraclass correlation*, or ICC.  This is a
distinct form of correlation from Pearson's correlation.  The ICC
takes on values from 0 to 1, with 1 corresponding to "perfect
clustering" -- the values within a cluster are identical, and 0
corresponding to "perfect independence" -- the mean value within each
cluster is identical across all the clusters.

We can assess ICC using two regression techniques, *marginal
regression*, and *multilevel regression*.  We will start by using a
technique called "Generalized Estimating Equations" (GEE) to fit
marginal linear models, and to estimate the ICC for the NHANES
clusters.

We will first look at the ICC for systolic blood pressure:

In [4]:
model = sm.GEE.from_formula("BPXSY1 ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.030


The estimated ICC is 0.03, which is small but not negligible.
Although an ICC is a type of correlation, its values are not directly
comparable to Pearson correlation values.  While 0.03 would generally
be considered to be very small as a Pearson correlation coefficient,
it is not especially small as an ICC.

To get a more systematic view of the ICC values induced by clustering
in these data, we calculate the ICC for a number of different
variables that appear in our analyses, either as outcomes or as
predictors.

In [5]:
# Recode smoking to a simple binary variable
da["smq"] = da.SMQ020.replace({2: 0, 7: np.nan, 9: np.nan})

for v in ["BPXSY1", "RIDAGEYR", "BMXBMI", "smq", "SDMVSTRA"]:
    model = sm.GEE.from_formula(v + " ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
    result = model.fit()
    print(v, result.cov_struct.summary())

BPXSY1 The correlation between two observations in the same cluster is 0.030
RIDAGEYR The correlation between two observations in the same cluster is 0.035


BMXBMI The correlation between two observations in the same cluster is 0.039
smq The correlation between two observations in the same cluster is 0.026


SDMVSTRA The correlation between two observations in the same cluster is 0.959


The values are generally similar to what we saw for blood pressure,
except for `SDMVSTRA`, which is one component of the cluster
definition itself, and therefore has a very high ICC.

To illustrate that the ICC values shown above are not consistent with
a complete absence of dependence, we simulate 10 sets of random data
and calculate the ICC value for each set:

In [6]:
for k in range(10):
    da["noise"] = np.random.normal(size=da.shape[0])
    model = sm.GEE.from_formula("noise ~ 1", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
    result = model.fit()
    print(v, result.cov_struct.summary())

SDMVSTRA The correlation between two observations in the same cluster is -0.002


SDMVSTRA The correlation between two observations in the same cluster is 0.000
SDMVSTRA The correlation between two observations in the same cluster is 0.001


SDMVSTRA The correlation between two observations in the same cluster is 0.001


SDMVSTRA The correlation between two observations in the same cluster is -0.002


SDMVSTRA The correlation between two observations in the same cluster is -0.002
SDMVSTRA The correlation between two observations in the same cluster is 0.001


SDMVSTRA The correlation between two observations in the same cluster is 0.001


SDMVSTRA The correlation between two observations in the same cluster is 0.001


SDMVSTRA The correlation between two observations in the same cluster is -0.000


We see that the estimated ICC for pure simulated noise is random but
highly concentrated near zero, varying from around `-0.002` to
`+0.002`.

## Conditional intraclass correlation

The ICC's studied above were *marginal*, in the sense that we were
looking at whether, say, the SBP values were more similar within
versus between clusters.  To the extent that such "cluster effects"
are found, it may be largely explained by demographic differences
among the clusters.  For example, we know from our previous analyses
with the NHANES data that older people have higher SBP than younger
people.  Also, some clusters may contain a slightly older or younger
set of people than others.  Thus, by controlling for age, we might
anticipate that the ICC will become smaller.  This is shown in the
next analysis:

In [7]:
model = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR", groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.019


The ICC for SBP drops from 0.03 to 0.02.  We can now assess whether it
drops even further when we add additional covariates that we know to
be predictive of blood pressure.

In [8]:
da["RIAGENDRx"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})

model = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result = model.fit()
print(result.cov_struct.summary())

The correlation between two observations in the same cluster is 0.013


The variable
[RIDRETH1](https://wwwn.cdc.gov/Nchs/Nhanes/2015-2016/DEMO_I.htm#RIDRETH1)
is a categorical variable containing 5 levels of race/ethnicity
information.  Since NHANES variables are coded numerically,
Statsmodels would have no way of knowing that these are codes and not
quantitative data, thus we must use the `C()` syntax in the formula
above to force this variable to be treated as begin categorical.  We
see here that the ICC has further reduced, to 0.013, due to
controlling for these additional factors including ethnicity.

## Marginal linear models with dependent data

Above we focused on quantifying the dependence induced by clustering.
By understanding the clustering structure, we have gained additional
insight about the data that complements our understanding of the mean
structure.  Another facet of working with dependent data is that while
the mean structure (i.e. the regression coefficients) can be estimated
without considering the dependence structure of the data, the standard
errors and other statistics relating to uncertainty will be wrong when
we ignore dependence in the data.

To illustrate this, below we fit two models with the same mean
structure to the NHANES data.  The first is a multiple regression
model fit using "ordinary least squares" (the default method for
independent data).  The second is fit using GEE, which allows us to
account for the dependence in the data.

In [9]:
# Fit a linear model with OLS
model1 = sm.OLS.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           data=da)
result1 = model1.fit()

# Git a marginal linear model using GEE to handle dependent data
model2 = sm.GEE.from_formula("BPXSY1 ~ RIDAGEYR + RIAGENDRx + BMXBMI + C(RIDRETH1)",
           groups="group",
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result2 = model2.fit()

x = pd.DataFrame({"OLS_params": result1.params, "OLS_SE": result1.bse,
                  "GEE_params": result2.params, "GEE_SE": result2.bse})
x = x[["OLS_params", "OLS_SE", "GEE_params", "GEE_SE"]]
print(x)

                   OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept           91.736583  1.339378   92.168530  1.384309
RIAGENDRx[T.Male]    3.671294  0.453763    3.650245  0.454498
C(RIDRETH1)[T.2]     0.855488  0.819486    0.159296  0.767025
C(RIDRETH1)[T.3]    -1.796132  0.671954   -2.233280  0.760228
C(RIDRETH1)[T.4]     3.813314  0.732355    3.105654  0.881580
C(RIDRETH1)[T.5]    -0.455347  0.808948   -0.439831  0.813675
RIDAGEYR             0.478699  0.012901    0.474101  0.018493
BMXBMI               0.278015  0.033285    0.280205  0.038553


In the results above, we see that the point estimates are similar
between the OLS and GEE fits of the model, but the standard errors
tend to be larger in the GEE fit.  For example, the standard errors
for BMI and age are 20-40% larger in the GEE fit.  Since we know that
there is dependence in these data that is driven by clustering, the
OLS approach is not theoretically justified (the OLS parameter
estimates remain in meaningful, but the standard errors do not).  GEE
parameter estimates and standard errors are meaningful in the presence
of dependence, as long as the dependence is exclusively between
observations within the same cluster.

## Marginal logistic regression with dependent data

In [10]:
# Relabel the levels, convert rare categories to missing.
da["DMDEDUC2x"] = da.DMDEDUC2.replace({1: "lt9", 2: "x9_11", 3: "HS", 4: "SomeCollege",
                                       5: "College", 7: np.nan, 9: np.nan})

# Fit a basic GLM
model1 = sm.GLM.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)",
           family=sm.families.Binomial(), data=da)
result1 = model1.fit()
result1.summary()

# Fit a marginal GLM using GEE
model2 = sm.GEE.from_formula("smq ~ RIDAGEYR + RIAGENDRx + C(DMDEDUC2x)",
           groups="group", family=sm.families.Binomial(),
           cov_struct=sm.cov_struct.Exchangeable(), data=da)
result2 = model2.fit(start_params=result1.params)

x = pd.DataFrame({"OLS_params": result1.params, "OLS_SE": result1.bse,
                  "GEE_params": result2.params, "GEE_SE": result2.bse})
x = x[["OLS_params", "OLS_SE", "GEE_params", "GEE_SE"]]
print(x)

                             OLS_params    OLS_SE  GEE_params    GEE_SE
Intercept                     -2.305999  0.114308   -2.249820  0.140567
RIAGENDRx[T.Male]              0.909597  0.060167    0.908682  0.062342
C(DMDEDUC2x)[T.HS]             0.943364  0.089663    0.887965  0.095397
C(DMDEDUC2x)[T.SomeCollege]    0.832227  0.084361    0.771636  0.104449
C(DMDEDUC2x)[T.lt9]            0.266228  0.109183    0.321784  0.141327
C(DMDEDUC2x)[T.x9_11]          1.098561  0.106697    1.062149  0.138401
RIDAGEYR                       0.018257  0.001725    0.017416  0.001803
